In [16]:
import numpy as np
import pandas as pd


In [17]:
df = pd.read_csv("POWER_Regional_Monthly_2000_2022.csv", skiprows=11)
df.head()

,PARAMETER,YEAR,LAT,LON,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN
0,T2M,2000,-10.25,33.25,23.47,22.57,21.85,21.80,21.07,19.27,18.12,20.41,23.60,25.78,23.77,22.90,22.05
1,T2M,2000,-10.25,33.75,22.78,22.09,21.23,20.86,19.42,17.76,16.56,18.56,21.77,24.10,23.04,22.39,20.87
2,T2M,2000,-10.25,34.25,27.18,26.67,25.27,24.81,23.66,22.20,20.91,22.62,25.71,27.79,27.14,26.47,25.03
3,T2M,2000,-10.25,34.75,25.37,24.94,23.32,22.95,22.22,20.89,19.80,21.57,24.41,26.57,25.45,24.61,23.50
4,T2M,2000,-10.25,35.25,23.41,23.10,21.46,21.20,20.65,19.41,18.57,20.45,23.06,25.30,23.72,22.73,21.91


In [18]:
# Check the unique values in the PARAMETER column
unique_parameters = df['PARAMETER'].unique()
print("Unique parameters in the dataset:", unique_parameters)


Unique parameters in the dataset: ['T2M' 'GWETROOT' 'PRECTOTCORR']


In [19]:
# List of month columns
months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

# Melt the dataframe to unpivot it from a wide format to a long format
df_melted = df.melt(id_vars=['PARAMETER', 'YEAR', 'LAT', 'LON'], value_vars=months, var_name='MONTH', value_name='VALUE')

# Map month abbreviations to numbers for easier date handling
month_map = {month: idx+1 for idx, month in enumerate(months)}
df_melted['MONTH'] = df_melted['MONTH'].map(month_map)

# Create a 'Date' column
df_melted['Date'] = pd.to_datetime(df_melted[['YEAR', 'MONTH']].assign(DAY=1))

# Pivot the dataframe to get one row per date per location with columns for each parameter
df_pivot = df_melted.pivot_table(index=['Date', 'LAT', 'LON'], columns='PARAMETER', values='VALUE').reset_index()

# Reorder the columns to match the desired output
df_pivot = df_pivot[['Date', 'T2M', 'GWETROOT','PRECTOTCORR', 'LAT', 'LON']]

# Display the reshaped dataframe
print(df_pivot.head())

# Save the cleaned and reshaped data to a new CSV file
df_pivot.to_csv('cleaned_combined_dataset.csv', index=False)



PARAMETER       Date    T2M  GWETROOT  PRECTOTCORR    LAT    LON
0         2000-01-01  27.53      0.43         5.27 -16.75  33.25
1         2000-01-01  28.01      0.46         5.27 -16.75  33.75
2         2000-01-01  28.65      0.51         5.27 -16.75  34.25
3         2000-01-01  28.97      0.56         5.27 -16.75  34.75
4         2000-01-01  28.49      0.57         5.27 -16.75  35.25


In [20]:
import pandas as pd

# Load the dataset
df = pd.read_csv('cleaned_combined_dataset.csv')

# Ensure the date column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Filter the data for the specified years
df_2012_2022 = df[(df['Date'].dt.year >= 2012) & (df['Date'].dt.year <= 2022)]
df_2017_2022 = df[(df['Date'].dt.year >= 2017) & (df['Date'].dt.year <= 2022)]

# Function to compute the required statistics
def compute_statistics(data):
    grouped = data.groupby([data['Date'].dt.year, 'LAT', 'LON']).agg(
        T2M_AVERAGE=('T2M', 'mean'),
        T2M_MIN=('T2M', 'min'),
        T2M_MAX=('T2M', 'max'),
        GWETROOT_AVERAGE=('GWETROOT', 'mean'),
        GWETROOT_MIN=('GWETROOT', 'min'),
        GWETROOT_MAX=('GWETROOT', 'max'),
        PRECTOTCORR_AVERAGE=('PRECTOTCORR', 'mean'),
        PRECTOTCORR_MIN=('PRECTOTCORR', 'min'),
        PRECTOTCORR_MAX=('PRECTOTCORR', 'max')
    ).reset_index()
    return grouped

# Compute statistics for both periods
statistics_2012_2022 = compute_statistics(df_2012_2022)
statistics_2017_2022 = compute_statistics(df_2017_2022)

# Display the results
print("Statistics from 2012 to 2022:")
print(statistics_2012_2022.head())

print("\nStatistics from 2017 to 2022:")
print(statistics_2017_2022.head())

# Save the results to CSV files
statistics_2012_2022.to_csv('statistic_2012_2022.csv', index=False)
statistics_2017_2022.to_csv('statistic_2017_2022.csv', index=False)


Statistics from 2012 to 2022:
   Date    LAT    LON  T2M_AVERAGE  T2M_MIN  T2M_MAX  GWETROOT_AVERAGE  \
0  2012 -16.75  33.25    25.278333    20.36    29.30          0.420833   
1  2012 -16.75  33.75    25.475833    20.69    29.42          0.460000   
2  2012 -16.75  34.25    25.987500    21.37    30.13          0.519167   
3  2012 -16.75  34.75    26.227500    21.78    30.54          0.570000   
4  2012 -16.75  35.25    25.669167    21.39    30.02          0.581667   

   GWETROOT_MIN  GWETROOT_MAX  PRECTOTCORR_AVERAGE  PRECTOTCORR_MIN  \
0          0.39          0.53             1.845833              0.0   
1          0.43          0.55             2.197500              0.0   
2          0.47          0.64             2.021667              0.0   
3          0.51          0.71             2.373333              0.0   
4          0.51          0.74             2.549167              0.0   

   PRECTOTCORR_MAX  
0            10.55  
1            10.55  
2            10.55  
3            1

In [21]:
unique_parameters = df['LAT'].unique()
print("Unique parameters in the dataset:", unique_parameters)
num_unique_parameters = len(unique_parameters)

print("Num unique parameters in the dataset:", num_unique_parameters)

Unique parameters in the dataset: [-16.75 -16.25 -15.75 -15.25 -14.75 -14.25 -13.75 -13.25 -12.75 -12.25
 -11.75 -11.25 -10.75 -10.25  -9.75  -9.25]
Num unique parameters in the dataset: 16


In [22]:
unique_parameters = df['LON'].unique()
print("Unique parameters in the dataset:", unique_parameters)
num_unique_parameters = len(unique_parameters)
print("Num unique parameters in the dataset:", num_unique_parameters)

Unique parameters in the dataset: [33.25 33.75 34.25 34.75 35.25 35.75]
Num unique parameters in the dataset: 6


### The expected number of unique lat and lon pairs should be 96. 

In [23]:
# Assuming statistics_2012_2022 is your DataFrame

# Grouping by a pair of columns, for example, 'LAT' and 'LON'
grouped = statistics_2012_2022.groupby(['LAT', 'LON'])

# Iterate over the groups and save each group as a separate dataset
for group_name, group_data in grouped:
    # Create a dataset name based on group name
    dataset_name = f"-9.25,35.75.csv"  # Example: group_40.7128_-74.0060.csv
    
    # Save the group data to a CSV file
    group_data.to_csv(dataset_name, index=False)  # Assuming you don't want to save the index


# Define latitude and longitude values
latitude_values = [-16.75, -16.25, -15.75, -15.25, -14.75, -14.25, -13.75, -13.25, -12.75, -12.25,
                   -11.75, -11.25, -10.75, -10.25, -9.75, -9.25]
longitude_values = [33.25, 33.75, 34.25, 34.75, 35.25, 35.75]

# Grouping by a pair of columns, for example, 'LAT' and 'LON'
grouped = statistics_2012_2022.groupby(['LAT', 'LON'])

# Iterate over each latitude and longitude pair
for lat in latitude_values:
    for lon in longitude_values:
        # Filter the data for the current latitude and longitude pair
        filtered_data = grouped.get_group((lat, lon))
        
        # Create a dataset name based on the latitude and longitude pair
        dataset_name = f"{lat},{lon}.csv"
        
        # Save the filtered data to a CSV file
        filtered_data.to_csv(dataset_name, index=False)


import pandas as pd

# Define latitude and longitude values
latitude_values = [-16.75, -16.25, -15.75, -15.25, -14.75, -14.25, -13.75, -13.25, -12.75, -12.25,
                   -11.75, -11.25, -10.75, -10.25, -9.75, -9.25]
longitude_values = [33.25, 33.75, 34.25, 34.75, 35.25, 35.75]

# Grouping by a pair of columns, for example, 'LAT' and 'LON'
grouped = statistics_2012_2022.groupby(['LAT', 'LON'])

# Columns to keep in the resulting CSV files

# Iterate over each latitude and longitude pair
for lat in latitude_values:
    for lon in longitude_values:
        # Filter the data for the current latitude and longitude pair
        filtered_data = grouped.get_group((lat, lon))
        
        # Drop the 'Date' column
        filtered_data = filtered_data.drop(columns=['Date'])
        
        # Calculate column averages
        column_averages = filtered_data.mean()
        
        # Create a dataset name based on the latitude and longitude pair
        dataset_name = f"{lat},{lon}.csv"
        
        # Save the column averages to a CSV file
        column_averages.to_csv(dataset_name)


In [24]:
# Define latitude and longitude values
latitude_values = [-16.75, -16.25, -15.75, -15.25, -14.75, -14.25, -13.75, -13.25, -12.75, -12.25,
                   -11.75, -11.25, -10.75, -10.25, -9.75, -9.25]
longitude_values = [33.25, 33.75, 34.25, 34.75, 35.25, 35.75]

# Grouping by a pair of columns, for example, 'LAT' and 'LON'
grouped = statistics_2012_2022.groupby(['LAT', 'LON'])

# Initialize an empty list to store dictionaries representing rows
rows = []

# Iterate over each latitude and longitude pair
for lat in latitude_values:
    for lon in longitude_values:
        # Filter the data for the current latitude and longitude pair
        filtered_data = grouped.get_group((lat, lon))
        
        # Drop the 'Date' column
        filtered_data = filtered_data.drop(columns=['Date'])
        
        # Calculate column averages
        column_averages = filtered_data.mean()
        
        # Create a dictionary representing the row
        row_data = {'LAT': lat, 'LON': lon, **column_averages}
        
        # Append the row to the list
        rows.append(row_data)

# Create a DataFrame from the list of rows
result_df = pd.DataFrame(rows)

# Save the result DataFrame to a CSV file
result_df.to_csv('10_year_average.csv', index=False)


In [25]:
# Define latitude and longitude values
latitude_values = [-16.75, -16.25, -15.75, -15.25, -14.75, -14.25, -13.75, -13.25, -12.75, -12.25,
                   -11.75, -11.25, -10.75, -10.25, -9.75, -9.25]
longitude_values = [33.25, 33.75, 34.25, 34.75, 35.25, 35.75]

# Grouping by a pair of columns, for example, 'LAT' and 'LON'
grouped = statistics_2017_2022.groupby(['LAT', 'LON'])

# Initialize an empty list to store dictionaries representing rows
rows = []

# Iterate over each latitude and longitude pair
for lat in latitude_values:
    for lon in longitude_values:
        # Filter the data for the current latitude and longitude pair
        filtered_data = grouped.get_group((lat, lon))
        
        # Drop the 'Date' column
        filtered_data = filtered_data.drop(columns=['Date'])
        
        # Calculate column averages
        column_averages = filtered_data.mean()
        
        # Create a dictionary representing the row
        row_data = {'LAT': lat, 'LON': lon, **column_averages}
        
        # Append the row to the list
        rows.append(row_data)

# Create a DataFrame from the list of rows
result_df = pd.DataFrame(rows)

# Save the result DataFrame to a CSV file
result_df.to_csv('5_year_average.csv', index=False)


In [26]:
# Load your datasets
df_5year = pd.read_csv('5_year_average.csv')
df_10year = pd.read_csv('10_year_average.csv')

# Rename the columns with respective prefixes
df_5year.columns = ['LAT', 'LON'] + ['5_' + col for col in df_5year.columns if col not in ['LAT', 'LON']]
df_10year.columns = ['LAT', 'LON'] + ['10_' + col for col in df_10year.columns if col not in ['LAT', 'LON']]

# Concatenate the datasets row-wise
combined_df = pd.concat([df_5year, df_10year], ignore_index=True)

# Assuming there may be duplicate LAT/LON pairs, we need to group by LAT and LON
# Aggregate by taking the first non-NA value
grouped_df = combined_df.groupby(['LAT', 'LON'], as_index=False).first()

# Save the grouped dataset to a new CSV file
grouped_df.to_csv('grouped_averages.csv', index=False)

print(grouped_df)


      LAT    LON  5_T2M_AVERAGE  5_T2M_MIN  5_T2M_MAX  5_GWETROOT_AVERAGE  \
0  -16.75  33.25      25.058611  20.128333  29.885000            0.431389   
1  -16.75  33.75      25.314167  20.523333  30.000000            0.465694   
2  -16.75  34.25      25.890972  21.231667  30.718333            0.521528   
3  -16.75  34.75      26.010833  21.265000  31.070000            0.578472   
4  -16.75  35.25      25.371111  20.603333  30.548333            0.607639   
..    ...    ...            ...        ...        ...                 ...   
91  -9.25  33.75      18.454583  14.376667  21.413333            0.779861   
92  -9.25  34.25      18.207361  14.628333  21.071667            0.647917   
93  -9.25  34.75      17.743056  14.130000  20.281667            0.722917   
94  -9.25  35.25      19.505556  15.855000  22.330000            0.751944   
95  -9.25  35.75      22.860278  19.240000  26.291667            0.656250   

    5_GWETROOT_MIN  5_GWETROOT_MAX  5_PRECTOTCORR_AVERAGE  5_PRECTOTCORR_MI

In [30]:
# Load the dataset
df = pd.read_csv('crop_recommendation.csv')

# Filter for the specified crops
crops_of_interest = ['rice', 'maize', 'kidneybeans']
filtered_df = df[df['Crop'].isin(crops_of_interest)]

# Group by 'Crop' and compute average, min, and max for each column
grouped_df = filtered_df.groupby('Crop').agg({
    'Nitrogen': ['mean', 'min', 'max'],
    'Phosphorus': ['mean', 'min', 'max'],
    'Potassium': ['mean', 'min', 'max'],
    'Temperature': ['mean', 'min', 'max'],
    'Humidity': ['mean', 'min', 'max'],
    'pH': ['mean', 'min', 'max'],
    'Rainfall': ['mean', 'min', 'max']
}).reset_index()

# Flatten the MultiIndex columns
grouped_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in grouped_df.columns.values]

# Rename columns to match the desired format
grouped_df.columns = [
    'CROP',
    'N_AVERAGE', 'N_MIN', 'N_MAX',
    'P_AVERAGE', 'P_MIN', 'P_MAX',
    'K_AVERAGE', 'K_MIN', 'K_MAX',
    'TEMP_AVERAGE', 'TEMP_MIN', 'TEMP_MAX',
    'HUM_AVERAGE', 'HUM_MIN', 'HUM_MAX',
    'PH_AVERAGE', 'PH_MIN', 'PH_MAX',
    'RAIN_AVERAGE', 'RAIN_MIN', 'RAIN_MAX'
]

# Save the grouped dataset to a new CSV file
grouped_df.to_csv('crop_statistics.csv', index=False)

print(grouped_df)


          CROP  N_AVERAGE  N_MIN  N_MAX  P_AVERAGE  P_MIN  P_MAX  K_AVERAGE  \
0  kidneybeans      20.75      0     40      67.54     55     80      20.05   
1        maize      77.76     60    100      48.44     35     60      19.79   
2         rice      79.89     60     99      47.58     35     60      39.87   

   K_MIN  K_MAX  ...   TEMP_MAX  HUM_AVERAGE    HUM_MIN    HUM_MAX  \
0     15     25  ...  24.923601    21.605357  18.092240  24.969699   
1     15     25  ...  26.549864    65.092249  55.282204  74.829137   
2     35     45  ...  26.929951    82.272822  80.122675  84.969072   

   PH_AVERAGE    PH_MIN    PH_MAX  RAIN_AVERAGE    RAIN_MIN    RAIN_MAX  
0    5.749411  5.502999  5.998125    105.919778   60.275525  149.744103  
1    6.245190  5.513698  6.995844     84.766988   60.651715  109.751538  
2    6.425471  5.005307  7.868475    236.181114  182.561632  298.560117  

[3 rows x 22 columns]
